In [17]:
# 1. 라이브러리 임포트
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [21]:
file_path = './data.csv'
data = pd.read_csv(file_path)

data = pd.get_dummies(data, columns=['환자의 BI-RADS'], drop_first=True)

# 2. 데이터 전처리
# 결측치 처리 (예시: BMI에서 Missing 값을 평균값으로 대체)
data['체질량지수 (BMI)'] = pd.to_numeric(data['체질량지수 (BMI)'], errors='coerce')
data['체질량지수 (BMI)'] = data['체질량지수 (BMI)'].fillna(data['체질량지수 (BMI)'].mean())


# Yes No => True False
data['유방촬영술 비교 영상'] = data['유방촬영술 비교 영상'] == 'Yes'
data['유방 밀도'] = data['유방 밀도'] == 'Yes'
data['유방암 가족력'] = data['유방암 가족력'] == 'Yes'
data['현재 호르몬 치료 사용 여부'] = data['현재 호르몬 치료 사용 여부'] == 'Yes'
data['이진 지표'] = data['이진 지표'] == 'Yes'

data.head()

,유방촬영술 당시 나이,방사선 전문의의 평가,암 진단 여부(이진 표시),유방촬영술 비교 영상,유방 밀도,유방암 가족력,현재 호르몬 치료 사용 여부,이진 지표,유방 조직 검사 이력,필름 또는 디지털 유방촬영 여부,암 유형,체질량지수 (BMI),환자 연구 ID,환자의 BI-RADS_Extremely dense,환자의 BI-RADS_Heterogeneously dense,환자의 BI-RADS_Scattered fibroglandular densities
0,62,Negative,False,True,False,False,True,False,False,No cancer diagnosis,24.0235443,1,NaN,False,False,True
1,65,Negative,False,True,False,False,True,False,False,No cancer diagnosis,Missing,2,NaN,True,False,False
2,69,Needs additional imaging,False,True,False,False,True,False,False,No cancer diagnosis,29.0524292,3,NaN,False,False,True
3,64,Benign findings,False,True,False,False,True,False,False,No cancer diagnosis,Missing,4,NaN,False,False,True
4,63,Probably benign,False,True,False,False,True,True,False,No cancer diagnosis,33.7295227,5,NaN,False,False,True


In [22]:


# 범주형 데이터를 숫자로 변환 (Label Encoding)
label_encoder = LabelEncoder()

# 암 진단 여부(이진 표시) - 예측할 대상
data['암 진단 여부(이진 표시)'] = label_encoder.fit_transform(data['암 진단 여부(이진 표시)'])

# 다른 범주형 변수들에 대해 label encoding 처리 (필요한 경우에만)
categorical_columns = ['방사선 전문의의 평가', '유방 밀도', '유방암 가족력', '현재 호르몬 치료 사용 여부', '유방 조직 검사 이력', '필름 또는 디지털 유방촬영 여부', '암 유형']
for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col].astype(str))

# 3. 특성과 레이블로 데이터 분할
X = data.drop(['암 진단 여부(이진 표시)', '환자 연구 ID'], axis=1)  # 예측에 사용할 피처들
y = data['암 진단 여부(이진 표시)']  # 타겟 변수

# 훈련 데이터와 테스트 데이터로 분할 (80% 훈련, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. SVM 모델 학습
model = SVC(kernel='linear')  # 선형 커널을 사용
model.fit(X_train, y_train)

# 5. 예측 및 평가
y_pred = model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f'정확도: {accuracy:.2f}')

# 상세 평가 (Precision, Recall, F1-Score)
print(classification_report(y_test, y_pred))


정확도: 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7948
           1       0.17      0.02      0.03        52

    accuracy                           0.99      8000
   macro avg       0.58      0.51      0.52      8000
weighted avg       0.99      0.99      0.99      8000



In [23]:
print(y_train.head())

5485     0
14857    0
6897     0
17911    0
19124    0
Name: 암 진단 여부(이진 표시), dtype: int64
